<a href="https://colab.research.google.com/github/KentaroOriuchi/Basic-MNIST-Project-20260104/blob/main/BasicMNISTProject_20260104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

**Data Pipeline**

In [2]:
# 1. Data Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.13, ), (0.31,))
])

In [4]:
# 2. Load Dataset
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           download = True,
                                           transform = transform)

test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          download = True,
                                          transform = transform)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 480kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.62MB/s]


In [12]:
# 3. Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 1000, shuffle = False)

**Create Neural Network**

In [13]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x

**Device and Optimizer**

In [14]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

# Initialize model and move to device
model = MNISTClassifier().to(device)

# Loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

Using cuda


**Training**

In [28]:
def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Print every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * correct / total
            print(f'[{batch_idx * 64} / {60000}]'
            f'Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%')
            running_loss = 0.0


**Evaluation**

In [29]:
def evaluation(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return 100. * correct/total

**Putting It All Together**

In [31]:
# Training loop
num_epochs = 3
for epock in range(num_epochs):
    print(f'\nEpoch: {epock + 1}')
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluation(model, test_loader, device)
    print(f"Test Accuracy: {accuracy:.2f}%")


Epoch: 1
[6400 / 60000]Loss: 0.114 | Accuracy: 96.7%
[12800 / 60000]Loss: 0.090 | Accuracy: 97.0%
[19200 / 60000]Loss: 0.097 | Accuracy: 97.0%
[25600 / 60000]Loss: 0.095 | Accuracy: 97.0%
[32000 / 60000]Loss: 0.088 | Accuracy: 97.1%
[38400 / 60000]Loss: 0.093 | Accuracy: 97.1%
[44800 / 60000]Loss: 0.085 | Accuracy: 97.1%
[51200 / 60000]Loss: 0.097 | Accuracy: 97.1%
[57600 / 60000]Loss: 0.101 | Accuracy: 97.1%
Test Accuracy: 96.53%

Epoch: 2
[6400 / 60000]Loss: 0.065 | Accuracy: 98.1%
[12800 / 60000]Loss: 0.066 | Accuracy: 98.0%
[19200 / 60000]Loss: 0.072 | Accuracy: 97.9%
[25600 / 60000]Loss: 0.074 | Accuracy: 97.9%
[32000 / 60000]Loss: 0.077 | Accuracy: 97.8%
[38400 / 60000]Loss: 0.068 | Accuracy: 97.8%
[44800 / 60000]Loss: 0.072 | Accuracy: 97.8%
[51200 / 60000]Loss: 0.074 | Accuracy: 97.8%
[57600 / 60000]Loss: 0.069 | Accuracy: 97.8%
Test Accuracy: 97.44%

Epoch: 3
[6400 / 60000]Loss: 0.050 | Accuracy: 98.5%
[12800 / 60000]Loss: 0.063 | Accuracy: 98.3%
[19200 / 60000]Loss: 0.050 | 